In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.wrappers.scikit_learn import KerasRegressor


C:\Users\hareevarshan\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
dataset = pd.read_csv('football_Pvalue.csv')

In [5]:
dataset.head()

,Session_type,Player_position,Duration_mins,Load,RTT_minus_1,RTT,RTT_plus_1,distance_z1,distance_z3,distance_z4,...,MetabolicPowerZone3Distance,MetabolicPowerZone4Distance,MetabolicPowerZone5Distance,MetabolicPowerZone6Distance,MetabolicPowerZone2Time,MetabolicPowerZone4Time,MetabolicPowerZone5Time,MetabolicPowerZone6Time,HighIntensityBurstNumber,RPE
0,Pitch Session,6,80,320,0.77,0.79,0.77,374.85,41.58,42.89,...,60.34,53.01,44.53,11.61,174.2,18.5,12.0,2.6,2,4
1,Pitch Session,2,80,320,0.72,0.78,0.93,410.55,36.36,15.00,...,65.80,53.35,39.26,12.88,171.5,22.0,13.0,3.7,1,4
2,Pitch Session,2,80,320,0.68,0.81,0.78,1963.77,828.10,462.98,...,680.65,721.60,533.76,193.19,1068.7,217.6,133.4,41.1,1,4
3,Pitch Session,2,80,320,0.68,0.81,0.78,192.26,291.09,54.44,...,224.14,236.30,93.05,5.03,101.1,71.1,24.2,1.2,0,4
4,Pitch Session,2,80,320,0.68,0.81,0.78,411.36,118.90,32.45,...,37.09,72.40,58.73,11.82,126.0,22.4,16.0,3.1,0,4


In [6]:
X = dataset.iloc[:, :-1].values

In [7]:
X[0]

array(['Pitch Session', 6, 80, 320, 0.77, 0.79, 0.77, 374.85, 41.58,
       42.89, 7.57, 0.83, 6.05, 26.91, 6.19, 0.0, 53.05, 4.19, 4.98,
       29.96, 60.1, 160.3, 0.0, 0.0, 11, 14, 0, 4, 84, 126, 74, 539.81,
       169.01, 356.93, 282.91, 62.51, 146.15, 339.32, 3.82, 55, 14, 1, 0,
       64.0, 4.62, 49.87, 56.14, 720.71, 4.809375, 6.771875, 133.35,
       235.99, 60.34, 53.01, 44.53, 11.61, 174.2, 18.5, 12.0, 2.6, 2],
      dtype=object)

In [8]:
y = dataset['RPE'].values

In [9]:
labelencoder = LabelEncoder()
X[:, 0] = labelencoder.fit_transform(X[:, 0])

In [10]:
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()

In [11]:
X = X[:, 1:]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [13]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(16568, 63) (16568,) (4143, 63) (4143,)


In [14]:
regressor = Sequential()

In [15]:
regressor.add(Dense(units=30, kernel_initializer = 'uniform', activation = 'relu', input_dim=63))

In [16]:
regressor.add(BatchNormalization(momentum=0.99, epsilon=0.001))

In [17]:
regressor.add(Dense(units=30, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(BatchNormalization(momentum=0.99, epsilon=0.001))
regressor.add(Dropout(0.2))

In [18]:
regressor.add(Dense(units = 1))

In [19]:
regressor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30)                1920      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30)                120       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                930       
_________________________________________________________________
batch_normalization_2 (Batch (None, 30)                120       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 3,121
Trainable params: 3,001
Non-trainable params: 120
_________________________________________________________________


In [20]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [21]:
regressor.fit(x = X_train, y = y_train, epochs = 300, batch_size = 32,validation_data=(X_test, y_test))

Train on 16568 samples, validate on 4143 samples
Epoch 1/300
16568/16568 [==============================] - 2s 96us/step - loss: 17.7842 - val_loss: 2.8815
Epoch 2/300
16568/16568 [==============================] - 1s 42us/step - loss: 2.6346 - val_loss: 73.8656
Epoch 3/300
16568/16568 [==============================] - 1s 42us/step - loss: 2.0392 - val_loss: 17.2520
Epoch 4/300
16568/16568 [==============================] - 1s 42us/step - loss: 1.6925 - val_loss: 4.0044
Epoch 5/300
16568/16568 [==============================] - 1s 42us/step - loss: 1.4551 - val_loss: 1.2141
Epoch 6/300
16568/16568 [==============================] - 1s 42us/step - loss: 1.3301 - val_loss: 1.9669
Epoch 7/300
16568/16568 [==============================] - 1s 41us/step - loss: 1.1526 - val_loss: 3.4158
Epoch 8/300
16568/16568 [==============================] - 1s 41us/step - loss: 1.1705 - val_loss: 14.9066
Epoch 9/300
16568/16568 [==============================] - 1s 40us/step - loss: 1.0538 - val_loss: 

16568/16568 [==============================] - 1s 40us/step - loss: 0.4929 - val_loss: 0.6895
Epoch 78/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.4870 - val_loss: 0.3506
Epoch 79/300
16568/16568 [==============================] - 1s 40us/step - loss: 0.4529 - val_loss: 0.3181
Epoch 80/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.5027 - val_loss: 0.4370
Epoch 81/300
16568/16568 [==============================] - 1s 42us/step - loss: 0.5199 - val_loss: 0.9502
Epoch 82/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.4673 - val_loss: 1.0051
Epoch 83/300
16568/16568 [==============================] - 1s 42us/step - loss: 0.4455 - val_loss: 1.6280
Epoch 84/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.5284 - val_loss: 0.5792
Epoch 85/300
16568/16568 [==============================] - 1s 42us/step - loss: 0.4795 - val_loss: 0.7169
Epoch 86/300
16568/16568 [========================

16568/16568 [==============================] - 1s 39us/step - loss: 0.3876 - val_loss: 0.4679
Epoch 154/300
16568/16568 [==============================] - 1s 39us/step - loss: 0.4136 - val_loss: 1.0188
Epoch 155/300
16568/16568 [==============================] - 1s 40us/step - loss: 0.3874 - val_loss: 0.3321
Epoch 156/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.4019 - val_loss: 0.3324
Epoch 157/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.4249 - val_loss: 0.1852
Epoch 158/300
16568/16568 [==============================] - 1s 42us/step - loss: 0.3872 - val_loss: 0.4907
Epoch 159/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.4004 - val_loss: 0.3112
Epoch 160/300
16568/16568 [==============================] - 1s 40us/step - loss: 0.3789 - val_loss: 0.4978
Epoch 161/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.4017 - val_loss: 0.8182
Epoch 162/300
16568/16568 [===============

Epoch 229/300
16568/16568 [==============================] - 1s 45us/step - loss: 0.3653 - val_loss: 1.1730
Epoch 230/300
16568/16568 [==============================] - 1s 43us/step - loss: 0.3425 - val_loss: 0.1794
Epoch 231/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.3397 - val_loss: 0.2591
Epoch 232/300
16568/16568 [==============================] - 1s 44us/step - loss: 0.3410 - val_loss: 0.4114
Epoch 233/300
16568/16568 [==============================] - 1s 39us/step - loss: 0.3659 - val_loss: 0.2805
Epoch 234/300
16568/16568 [==============================] - 1s 44us/step - loss: 0.3766 - val_loss: 0.2267
Epoch 235/300
16568/16568 [==============================] - 1s 43us/step - loss: 0.3864 - val_loss: 0.7311
Epoch 236/300
16568/16568 [==============================] - 1s 42us/step - loss: 0.3689 - val_loss: 0.2296
Epoch 237/300
16568/16568 [==============================] - 1s 41us/step - loss: 0.3725 - val_loss: 0.2370
Epoch 238/300
16568/16568 [=

In [22]:
prediction = regressor.evaluate(x = X_test, y = y_test)
print ("Loss = " + str(prediction))

4143/4143 [==============================] - 0s 22us/step
Loss = 0.3392350407411245


In [23]:
y_pred = regressor.predict(X_test)

In [24]:
y_pred[0:10]

array([[ 7.7796683 ],
       [ 8.055178  ],
       [ 7.1084857 ],
       [ 8.106271  ],
       [ 5.0269976 ],
       [ 5.2747087 ],
       [-0.74293184],
       [ 8.939023  ],
       [ 4.1025095 ],
       [ 7.2232223 ]], dtype=float32)

In [25]:
mean_squared_error(y_test, y_pred)

0.3392350429091212

In [26]:
r2_score(y_test, y_pred)

0.9175682879008149

In [27]:
from sklearn.metrics import explained_variance_score

In [28]:
explained_variance_score(y_test, y_pred)

0.9185690149011076

In [29]:
from sklearn.metrics import median_absolute_error

In [30]:
median_absolute_error(y_test, y_pred)

0.13880491256713867